In [33]:
import pandas as pd
import json
import re
from collections import defaultdict

a = input("In: ")
b = input("Out: ")

df = pd.read_csv(a+'.csv', sep='\t', header=None,
                 index_col=0, engine='python', encoding='utf-8')
df = df.rename(columns={1:'full_name', 2:'grade', 3:'publisher',
                        4:'subject', 5:'standart', 6:'lvl'})
df['book_name'] = ''
df['authors'] = ''
df['edit_by'] = ''

author_name = re.compile(r'[А-ЯЁ][А-ЯЁа-яё-]+\s[А-ЯЁ.]{2,4}')
item_name = re.compile(r'[А-ЯЁA-Z][\w\W]+')

for ind, row in df.iterrows():
    full_name = row['full_name']
    
    if '/' in full_name and not ' / ' in full_name:
        full_name = full_name.replace('/', ' / ')
    
    if '  ' in full_name:
        full_name = full_name.replace('  ', ' ')
    
    if "Под ред." in full_name:
        split_name = full_name.split('Под ред')
        edit_by = re.findall(author_name, split_name[1]) #yeah
        authors = re.findall(author_name, split_name[0]) #yeah
        wet_name = re.sub(author_name, '', split_name[1].replace('Под ред', ''))
        book_name = re.search(item_name, wet_name).group() #yeah
        row['book_name'] = book_name
        row['authors'] = ', '.join(authors)
        row['edit_by'] = ', '.join(edit_by)
    else:
        authors = re.findall(author_name, full_name) #yeah
        wet_name = re.sub(author_name, '', full_name)
        book_name = re.search(item_name, wet_name).group() #yeah    
        row['book_name'] = book_name
        row['authors'] = ', '.join(authors)
            
    row['full_name'] = full_name
        
dd = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for indx, row in df.iterrows():
    dictus_bigus = {'full_name':row['full_name'], 'book_name':row['book_name'], 
                    'authors':row['authors'].split(', '), 'edit_by':row['edit_by'].split(', '), 
                    'grade':row['grade'], 'publisher':row['publisher'], 'indx':indx}
    dd[row['lvl']][row['standart']][row['subject']].append(dictus_bigus)
    
neu_Ordnung = ['lvl', 'grade', 'subject', 'book_name', 'authors', 
               'edit_by', 'full_name' , 'publisher', 'standart']

df[neu_Ordnung].to_csv(b+'.csv', sep='\t', encoding='utf-8', 
                       index='0', index_label='index')

with open(b+'.json', 'w', encoding='utf-8') as f:
    json.dump(dd, f, ensure_ascii=False)